# Token classification (PyTorch)

**Instalação de pacotes e configuração inicial**

Esses comandos instalam as bibliotecas necessárias: datasets (para carregar os dados), evaluate (para calcular as métricas de desempenho), transformers (para modelos pré-treinados e ferramentas associadas), accelerate (para otimização de treinamento) e git-lfs (para manipulação de grandes arquivos com o Git).

In [138]:
!pip install datasets evaluate transformers[sentencepiece]
!pip install accelerate
# To run the training on TPU, you will need to uncomment the following line:
# !pip install cloud-tpu-client==0.10 torch==1.9.0 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl
!apt install git-lfs

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


**Configuração do Git e Hugging Face**

Esses comandos configuram o Git para garantir que as alterações sejam atribuídas corretamente ao seu repositório do Hugging Face e fazem login no Hugging Face Hub para enviar modelos e dados treinados.

In [139]:
!git config --global user.email "adrianomartins202486@gmail.com"
!git config --global user.name "Adriano2024"

In [140]:
from huggingface_hub import notebook_login

notebook_login()

**Carregamento e inspeção dos dados**


Aqui, o conjunto de dados CoNLL-2003, um popular conjunto de dados para NER, é carregado. O conjunto contém três divisões: treino, validação e teste.

In [143]:
from datasets import load_dataset

raw_datasets = load_dataset("conll2003")

**Visualização dos dados e preparação**

Isso mostra os tokens e as etiquetas de entidade do primeiro exemplo no conjunto de treino.

In [144]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})

In [145]:
raw_datasets["train"][0]["tokens"]

['EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'lamb', '.']

In [146]:
raw_datasets["train"][0]["ner_tags"]

[3, 0, 7, 0, 0, 0, 7, 0, 0]

In [147]:
ner_feature = raw_datasets["train"].features["ner_tags"]
ner_feature

Sequence(feature=ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC'], id=None), length=-1, id=None)

In [148]:
label_names = ner_feature.feature.names
label_names

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

In [149]:
words = raw_datasets["train"][0]["tokens"]
labels = raw_datasets["train"][0]["ner_tags"]
line1 = ""
line2 = ""
for word, label in zip(words, labels):
    full_label = label_names[label]
    max_length = max(len(word), len(full_label))
    line1 += word + " " * (max_length - len(word) + 1)
    line2 += full_label + " " * (max_length - len(full_label) + 1)

print(line1)
print(line2)

EU    rejects German call to boycott British lamb . 
B-ORG O       B-MISC O    O  O       B-MISC  O    O 


In [150]:
from transformers import AutoTokenizer

model_checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [151]:
tokenizer.is_fast

True

In [152]:
inputs = tokenizer(raw_datasets["train"][0]["tokens"], is_split_into_words=True)
inputs.tokens()

['[CLS]',
 'EU',
 'rejects',
 'German',
 'call',
 'to',
 'boycott',
 'British',
 'la',
 '##mb',
 '.',
 '[SEP]']

In [153]:
inputs.word_ids()

[None, 0, 1, 2, 3, 4, 5, 6, 7, 7, 8, None]

**Alinhamento de rótulos com tokens**

Essa função alinha as etiquetas de entidade com os tokens gerados pelo tokenizador. Ela garante que as entidades sejam corretamente associadas aos tokens divididos.

In [154]:
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # Start of a new word!
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # Special token
            new_labels.append(-100)
        else:
            # Same word as previous token
            label = labels[word_id]
            # If the label is B-XXX we change it to I-XXX
            if label % 2 == 1:
                label += 1
            new_labels.append(label)

    return new_labels

In [155]:
labels = raw_datasets["train"][0]["ner_tags"]
word_ids = inputs.word_ids()
print(labels)
print(align_labels_with_tokens(labels, word_ids))

[3, 0, 7, 0, 0, 0, 7, 0, 0]
[-100, 3, 0, 7, 0, 0, 0, 7, 0, 0, 0, -100]


In [156]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True
    )
    all_labels = examples["ner_tags"]
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))

    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

**Tokenização e transformação dos dados**

Aqui, o conjunto de dados é tokenizado usando a função tokenize_and_align_labels, que aplica o tokenizador a cada exemplo e alinha as etiquetas com os tokens.

In [157]:
tokenized_datasets = raw_datasets.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=raw_datasets["train"].column_names,
)

Map:   0%|          | 0/3250 [00:00<?, ? examples/s]

In [158]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [159]:
batch = data_collator([tokenized_datasets["train"][i] for i in range(2)])
batch["labels"]

tensor([[-100,    3,    0,    7,    0,    0,    0,    7,    0,    0,    0, -100],
        [-100,    1,    2, -100, -100, -100, -100, -100, -100, -100, -100, -100]])

In [160]:
for i in range(2):
    print(tokenized_datasets["train"][i]["labels"])

[-100, 3, 0, 7, 0, 0, 0, 7, 0, 0, 0, -100]
[-100, 1, 2, -100]


In [161]:
!pip install seqeval

**Avaliação e Métricas**

O código carrega a métrica seqeval para avaliar o desempenho do modelo em tarefas de NER, como precisão, recall, F1 e acurácia.

In [162]:
import evaluate

metric = evaluate.load("seqeval")

In [163]:
labels = raw_datasets["train"][0]["ner_tags"]
labels = [label_names[i] for i in labels]
labels

['B-ORG', 'O', 'B-MISC', 'O', 'O', 'O', 'B-MISC', 'O', 'O']

In [164]:
predictions = labels.copy()
predictions[2] = "O"
metric.compute(predictions=[predictions], references=[labels])

{'MISC': {'precision': 1.0,
  'recall': 0.5,
  'f1': 0.6666666666666666,
  'number': 2},
 'ORG': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'overall_precision': 1.0,
 'overall_recall': 0.6666666666666666,
 'overall_f1': 0.8,
 'overall_accuracy': 0.8888888888888888}

In [165]:
import numpy as np


def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": all_metrics["overall_precision"],
        "recall": all_metrics["overall_recall"],
        "f1": all_metrics["overall_f1"],
        "accuracy": all_metrics["overall_accuracy"],
    }

In [166]:
id2label = {i: label for i, label in enumerate(label_names)}
label2id = {v: k for k, v in id2label.items()}

**Preparação do modelo e treinamento**

O modelo BERT (bert-base-cased) é carregado com uma camada de classificação de tokens. O id2label e label2id são mapeamentos entre os IDs das etiquetas e as etiquetas de texto.

In [167]:
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    id2label=id2label,
    label2id=label2id,
)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [168]:
model.config.num_labels

9

In [169]:
from huggingface_hub import notebook_login

notebook_login()

**Configuração do treinamento**

Aqui, são definidos os parâmetros de treinamento, incluindo a taxa de aprendizado, o número de épocas e a estratégia de salvamento e avaliação.

In [170]:
from transformers import TrainingArguments

args = TrainingArguments(
    "bert-finetuned-ner",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=True,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


**Treinamento com o Trainer**

O objeto Trainer gerencia o processo de treinamento. Ele usa o conjunto de dados tokenizado, o modelo, as métricas e o tokenizador definidos anteriormente. O treinamento é iniciado com o método train().

In [171]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)
trainer.train()

<ipython-input-171-ddd9b7f8600f>:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.075000,0.072124,0.901076,0.930495,0.915549,0.979882
2,0.035100,0.067811,0.930109,0.945136,0.937563,0.983958
3,0.021200,0.060965,0.933212,0.950017,0.941539,0.986107


TrainOutput(global_step=5268, training_loss=0.06584265273723443, metrics={'train_runtime': 602.5158, 'train_samples_per_second': 69.912, 'train_steps_per_second': 8.743, 'total_flos': 920771584279074.0, 'train_loss': 0.06584265273723443, 'epoch': 3.0})

**Push para Hugging Face Hub**

Ao final do treinamento, o modelo é enviado para o Hugging Face Hub, onde pode ser acessado ou compartilhado.


In [33]:
trainer.push_to_hub(commit_message="Training complete")

CommitInfo(commit_url='https://huggingface.co/Adriano2024/bert-finetuned-ner/commit/c910fa1dd5597313d79fa49870d8a7c595891600', commit_message='Training complete', commit_description='', oid='c910fa1dd5597313d79fa49870d8a7c595891600', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Adriano2024/bert-finetuned-ner', endpoint='https://huggingface.co', repo_type='model', repo_id='Adriano2024/bert-finetuned-ner'), pr_revision=None, pr_num=None)

In [34]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    tokenized_datasets["train"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=8,
)
eval_dataloader = DataLoader(
    tokenized_datasets["validation"], collate_fn=data_collator, batch_size=8
)

In [35]:
model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    id2label=id2label,
    label2id=label2id,
)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [118]:
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=2e-5)

In [119]:
from accelerate import Accelerator

accelerator = Accelerator()
model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader
)

In [120]:
from transformers import get_scheduler

num_train_epochs = 3
num_update_steps_per_epoch = len(train_dataloader)
num_training_steps = num_train_epochs * num_update_steps_per_epoch

lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

In [128]:
from huggingface_hub import snapshot_download
snapshot_download(repo_id="Alexander-Learn/bert-finetuned-ner-accelerate", local_dir="https://huggingface.co/Adriano2024/bert-finetuned-ner-accelerate")


Fetching 7 files:   0%|          | 0/7 [00:00<?, ?it/s]

'/content/https:/huggingface.co/Adriano2024/bert-finetuned-ner-accelerate'

In [125]:
import shutil
shutil.rmtree("/content/bert-finetuned-ner-accelerate", ignore_errors=True)

In [131]:
from huggingface_hub import Repository, get_full_repo_name

model_name = "bert-finetuned-ner"
repo_name = get_full_repo_name(model_name)
repo_name

'Adriano2024/bert-finetuned-ner'

In [132]:
output_dir = "bert-finetuned-ner-accelerate"
repo = Repository(output_dir, clone_from=repo_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'Repository' (from 'huggingface_hub.repository') is deprecated and will be removed from version '1.0'. Please prefer the http-based alternatives instead. Given its large adoption in legacy code, the complete removal is only planned on next major release.
For more details, please read https://huggingface.co/docs/huggingface_hub/concepts/git_vs_http.
  warnings.warn(warning_message, FutureWarning)
Cloning https://huggingface.co/Adriano2024/bert-finetuned-ner into local empty directory.


Download file model.safetensors:   0%|          | 8.00k/411M [00:00<?, ?B/s]

Download file runs/Nov13_16-24-52_d5e5fcfc049d/events.out.tfevents.1731515104.d5e5fcfc049d.241.0: 100%|#######…

Clean file runs/Nov13_16-24-52_d5e5fcfc049d/events.out.tfevents.1731515104.d5e5fcfc049d.241.0:  11%|#1        …

Download file training_args.bin: 100%|##########| 5.18k/5.18k [00:00<?, ?B/s]

Clean file training_args.bin:  19%|#9        | 1.00k/5.18k [00:00<?, ?B/s]

Clean file model.safetensors:   0%|          | 1.00k/411M [00:00<?, ?B/s]

In [133]:
def postprocess(predictions, labels):
    predictions = predictions.detach().cpu().clone().numpy()
    labels = labels.detach().cpu().clone().numpy()

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    return true_labels, true_predictions

In [134]:
from tqdm.auto import tqdm
import torch

progress_bar = tqdm(range(num_training_steps))

for epoch in range(num_train_epochs):
    # Training
    model.train()
    for batch in train_dataloader:
        outputs = model(**batch)
        loss = outputs.loss
        accelerator.backward(loss)

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

    # Evaluation
    model.eval()
    for batch in eval_dataloader:
        with torch.no_grad():
            outputs = model(**batch)

        predictions = outputs.logits.argmax(dim=-1)
        labels = batch["labels"]

        # Necessary to pad predictions and labels for being gathered
        predictions = accelerator.pad_across_processes(predictions, dim=1, pad_index=-100)
        labels = accelerator.pad_across_processes(labels, dim=1, pad_index=-100)

        predictions_gathered = accelerator.gather(predictions)
        labels_gathered = accelerator.gather(labels)

        true_predictions, true_labels = postprocess(predictions_gathered, labels_gathered)
        metric.add_batch(predictions=true_predictions, references=true_labels)

    results = metric.compute()
    print(
        f"epoch {epoch}:",
        {
            key: results[f"overall_{key}"]
            for key in ["precision", "recall", "f1", "accuracy"]
        },
    )

    # Save and upload
    accelerator.wait_for_everyone()
    unwrapped_model = accelerator.unwrap_model(model)
    unwrapped_model.save_pretrained(output_dir, save_function=accelerator.save)
    if accelerator.is_main_process:
        tokenizer.save_pretrained(output_dir)
        repo.push_to_hub(
            commit_message=f"Training in progress epoch {epoch}", blocking=False
        )

  0%|          | 0/5268 [00:00<?, ?it/s]

epoch 0: {'precision': 0.9347021204981488, 'recall': 0.9038242473555737, 'f1': 0.9190038884752213, 'accuracy': 0.9812798022016836}
epoch 1: {'precision': 0.9404240996297543, 'recall': 0.918625678119349, 'f1': 0.9293970893970893, 'accuracy': 0.9833107670571614}
epoch 2: {'precision': 0.9488387748232918, 'recall': 0.9292895994725564, 'f1': 0.9389624448330418, 'accuracy': 0.9858568316948254}


In [135]:
accelerator.wait_for_everyone()
unwrapped_model = accelerator.unwrap_model(model)
unwrapped_model.save_pretrained(output_dir, save_function=accelerator.save)

**Pipeline de Inferência e o Teste de uma frase**

Depois de treinar o modelo, um pipeline de inferência é criado para realizar a NER em novos textos. O modelo é carregado para uma GPU, se disponível, para acelerar a inferência e o modelo é usado para identificar entidades em uma frase de exemplo. O resultado é exibido em formato de tabela, com as entidades, tipos, pontuação e posições.

In [188]:
from transformers import pipeline
import pandas as pd

# Se a GPU estiver disponível, o modelo será carregado nela
device = 0  # '0' para a primeira GPU (se houver mais de uma, '1' seria a segunda GPU, e assim por diante)
nlp = pipeline('ner', device=device)

# Usando um modelo público do Hugging Face
model_checkpoint = "dbmdz/bert-large-cased-finetuned-conll03-english"  # Modelo disponível publicamente
token_classifier = pipeline(
    "token-classification", model=model_checkpoint, aggregation_strategy="simple"
)

# Executar a análise de entidades
result = token_classifier("Elon Musk, the CEO of SpaceX, moved from Pretoria, South Africa, to the United States to work on his Tesla project.")

# Criar uma lista de dicionários com os dados formatados para a tabela
data = []
for entity in result:
    data.append({
        'Entity': entity['word'],
        'Type': entity['entity_group'],
        'Score': round(entity['score'], 4),
        'Start': entity['start'],
        'End': entity['end']
    })

# Criar um DataFrame do pandas para exibir em formato de tabela
df = pd.DataFrame(data)

# Exibir a tabela
df


No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english and revision 4c53496 (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkp

,Entity,Type,Score,Start,End
0,Elon Musk,PER,0.9989,0,9
1,SpaceX,ORG,0.9992,22,28
2,Pretoria,LOC,0.9991,41,49
3,South Africa,LOC,0.9998,51,63
4,United States,LOC,0.9997,72,85
5,Tesla,ORG,0.6201,101,106
